In [ ]:
# Dzień piąty DWthon 👋 - Hack Outside the Box 

### Celem jest poznanie, czym jest tak zwany "aha moment"!

Czy wiesz co to jest "aha-moment"? To jest taki moment, kiedy mówisz... "aha" 😁. Czyli coś Cię olśniło i rzeczywistość (a przynajmniej jej kawałek) stał się bardziej dla Ciebie zrozumiały.

O "aha-momencie" można długo mówić, lepiej podam Ci kilka przykładów.


### Facebook (7 przyjaciół w 10 dni)
Jeśli użytkownik dodaje co najmniej **10 znajomych** w ciągu **pierwszych 7 dni** od rejestracji, wtedy ten użytkownik będzie bardziej zaangażowany i na dłużej zostanie w Facebook.

### Dropbox (1 plik, 1 folder, 1 urządzenie)
Jeśli użytkownik doda co najmniej jeden plik, jeden folder i zsynchronizuje się z co najmniej jednym urządzeniem, to wtedy ten użytkownik zostanie na dłużej.

### LinkedIn - X połączeń w Y dni


To są proste reguły, które potrafią opisać najważniejsze, co ma się wydarzyć w firmie, aby firma rosła. Innymi słowy to jest stosowanie wprost reguły 80/20.


Trudność polega na tym, że aby znaleźć tę prostą regułę, to... no właśnie, jak można ją znaleźć? Przecież takich prostych reguł może być tysiące lub nawet miliony, zabraknie nam czasu, aby je wszystkie sprawdzić. Tutaj na pomoc nam może przyjść `Machine Learning`.

### Zanim zaczniemy | ważne ❗

Ankieta numer 5! Każdy dzień DWthon - hack outside the box ma dedykowaną ankietę. Wypełnij ją zanim przejdziesz do nauki. Zajmie Ci to tylko 3 minutki ;) 

### [Wypełnij ankietę](https://bit.ly/3qxZd9S)  i mierz deltę swego rozwoju :) 

### Gdzie zadawać pytania ❓ 

Jeśli napotkasz trudności podczas wykonywania zadań z tego notebooka, to koniecznie napisz w kanale [dwthon_day5](https://bit.ly/3qEBEMv)

Pamiętaj, aby szczegółowo doprecyzować, z czym masz problem. Najlepiej wrzuć screen z kodem swoim lub błędem, który widzisz i napisz, którego zadania dotyczy :)

%%html
<iframe style="height:500px;width:100%" src="https://www.youtube.com/embed/EeFJLSS3_x0" frameborder="0" allow="autoplay; encrypted-media" allowfullscreen></iframe>

### Krok po kroku 

Nagrałem dla Ciebie także materiał wideo "krok po kroku". W wideo poniżej znajduje się dokładnie to, co w tym notebooku tylko tłumaczę wszystko, aby ułatwić Ci pracę i zrozumienie zagadnień i zadań :) Obejrzyj, jeśli potrzebujesz takiego dodatkowego wsparcia. 

%%html
<iframe style="height:500px;width:100%" src="https://www.youtube.com/embed/7UTzlVkK17k" frameborder="0" allow="autoplay; encrypted-media" allowfullscreen></iframe>

## Biblioteki
Potrzebujemy `pandas` i jeszcze trochę więcej innych bibliotek. 
* `DecisionTreeClassifier` - do trenowania modelu
* `sklearn.model_selection` walidacja modelu
* `gc` do sprzątania w pamięci RAM

Dodatkowo użyjemy nowego bardziej zaawansowanego modelu `xgboost` oraz biblioteki `eli5` do podglądania ważności cech.
`from collections import Counter` przyda nam się do wygodnego zliczenia liczby wystąpień (np. ile razy klient kupił produkt x).

import pandas as pd
import numpy as np

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
import xgboost as xgb
import eli5 
from collections import Counter

import gc

## Wczytajmy dane

df = pd.read_hdf("../input/data.h5")

print(df.shape)
df.sample(5)

## Przygotowujemy dane


To jest kod z poprzedniej lekcji. Jedynie poukładamy kod w funkcji, aby łatwiej było nam eksperymentować później.

df_customers = (
    df[ ["price_total", "customer_id"] ]
    .groupby("customer_id")
    .agg("sum")
    .reset_index()
    .sort_values(by="price_total", ascending=False)
    .rename(columns={"price_total": "customer_price_total"})
)


df_customers["cumsum"] = df_customers["customer_price_total"].cumsum()
value_80prc = int(df["price_total"].sum() * 0.8)
df_customers["most_revenue_customer"] = df_customers["cumsum"] < value_80prc


top_customers = set(df_customers[ df_customers["most_revenue_customer"] ]["customer_id"].unique())

del df_customers
gc.collect()

def feature_engineering(df):
    df_customers = (
        df
        .groupby("customer_id")
        .agg(
            count_orders=("order_id", lambda x: len(set(x))),
            count_unq_products=("product_id", lambda x: len(set(x))),
            sum_quantity=("quantity", np.sum),
            sum_price_unit=("price_unit", np.sum),
            sum_price_total=("price_total", np.sum),
            count_unq_countries=("country_id", lambda x: len(set(x))),
            prob_canceled=("is_canceled", np.mean)
        ).reset_index()
    )
    
    
    
    return df_customers


def get_feats(df_customers, black_list=["most_revenue_customer"]):
    feats = list(df_customers.select_dtypes([np.number, bool]).columns)
    return [x for x in feats if x not in black_list]

def get_X_y(df_customers, top_customers, feats):
    df_customers["most_revenue_customer"] = df_customers["customer_id"].map(lambda x: x in top_customers)
    
    X = df_customers[feats].values
    y = df_customers["most_revenue_customer"].values
    
    return X, y


def train_and_get_scores(model, X, y, scoring="accuracy", cv=5):

    scores = cross_val_score(model, X, y, scoring=scoring, cv=cv)
    return np.mean(scores), np.std(scores)

## Model 1

☝️ Dokładnie to, co zrobiliśmy ostatnio.

df_customers = feature_engineering(df)
feats = get_feats(df_customers)
X, y = get_X_y(df_customers, top_customers, feats)
model = DecisionTreeClassifier(max_depth=5)

train_and_get_scores(model, X, y)

Mamy wynik 99.9%.

Można się cieszyć, ale... tak naprawdę opowiem Ci o jednym mechanizmie, który regularnie stosuję. Nazywam go wahadło.

Idea polega na tym, aby najpierw być dużym optymistą i bardzo upraszczać rzeczywistość na początku. 
Następnie, jak już mamy dobry wynik, to wchodzimy w "zupełnie inne buty". 
Wchodzimy w rolę aktywnego "krytyka", który próbuje zrobić wszystko, aby udowodnić, że to, co zrobiliśmy nie ma sensu.
Powtarzamy tak w kółko. Robimy kilka iteracji :)
Najważniejsze jest to, że dzięki przełączaniu się pomiędzy rolami, można bardzo szybko biec od przodu, ale z drugiej 
strony mieć poczucie, że to nadal ma sens!

## 🧠 Włączmy myślenie krytycznie

Na ten moment wykonaliśmy pierwszy krok, czyli optymistyczny i dostaliśmy wynik 99%. 
Bardzo dobrze, ale... Problem polega na tym, że `price_unit_total`jest zbyt mocną cechą. 
Czyli ta cecha wprost mówi, czy klient należy do segmentu `top_customers` czy nie. 
Dlatego teraz musimy co najmniej tę cechę wyłączyć i sprawdzić, jak zachowa się model.

Wystarczy dodać `sum_price_total` do listy, którą będziemy ignorować. 

Zobacz 👀.


## Model 2

df_customers = feature_engineering(df)
feats = get_feats(df_customers, black_list=["most_revenue_customer", "sum_price_total"])
X, y = get_X_y(df_customers, top_customers, feats)
model = DecisionTreeClassifier(max_depth=5)

train_and_get_scores(model, X, y)

Jak widzisz, jakość modelu nam spadła do 77%. Z jednej strony smutna sprawa 😂, ale z drugiej strony używamy modelu ML, aby znajdował mniej oczywiste zależności. Bo te oczywiste człowiek sam łatwo znajdzie sam 🤦‍♂️.


### Myślenie optymistyczne
To teraz wróćmy znów do bycia optymistą. Pamiętasz na czym polega zasada wahadła? Wahamy się tam i z powrotem ;) 


Można zmienić model na bardziej zaawansowany i to powinno poprawić wynik. Użyjmy `xgboost`.
Ta nazwa może Ci nic nie mówić i to jest ok na tym etapie 👌.

## Model 3

Zwróć uwagę, że zmieniła się ta linijka: `model = xgb.XGBClassifier(max_depth=5, n_estimators=50, learning_rate=0.3)`.

df_customers = feature_engineering(df)
feats = get_feats(df_customers, black_list=["most_revenue_customer", "sum_price_total"])
X, y = get_X_y(df_customers, top_customers, feats)
model = xgb.XGBClassifier(max_depth=5, n_estimators=50, learning_rate=0.3)

train_and_get_scores(model, X, y)

## Mamy wynik 81%.

Spróbujmy zrobić pierwszą interpretację modelu i zobaczyć, co wg nieg0 jest ważne.

model.fit(X, y)

eli5.show_weights(model, feature_names=feats)

### Wstępnie wnioski:
#### TOP3 
- `sum_quantity` brzmi jako najważniejsza cecha*
- `count_orders`
- `sum_price_unit`

Wyglada na to, że cecha `count_unq_countries` jest mało przydatna.


☝️ Trzeba uważać na to, co jest "ważne", bo doświadczenie pokazuje, że to może być dość zmienne i w tej zmienności
trzeba nauczyć się poruszać.


## Następny krok

Możemy teraz trochę pofilozofować nad tymi kilkoma cechami, ale to jest mało istotne. 
Te cechy same w sobie są dość oczywiste. Nie ma poczucia "dużej" wartości dodanej. 

Ciekawostką okazało się, że `sum_quantity` jest istotna. Tylko tutaj też nie ma dużego zaskoczenia.

Wygenerujmy zatem więcej cech, niech model znajdzie na co warto zwrócić naszą uwagę :) 


## Produkty


Możemy dodać dla każdego produktu osobną kolumnę i jako wartość w tej kolumnie dodać,
ile produktów już było kupionych (czyli suma).
W ten sposób chcemy znaleźć, które produkty są "najciekawsze", warte tego, aby zwrócić na nie uwagę.

☝️ Oczywiście możemy sprawdzić każdy produkt manualnie, ale mamy ich ponad 3.8k, może to nam trochę zająć czasu. 
Użyjmy lepiej do tego ML, będzie szybciej, sprawniej i efektywniej.

Tylko najpierw trzeba przygotować dane. Zmieńmy naszą funkcję `feature_engineering`. 

def feature_engineering(df):
    
    def counter(vals):
        cntr = Counter()
        cntr.update(vals)
        return cntr
    
    df_customers = (
        df
        .groupby("customer_id")
        .agg(
            count_orders=("order_id", lambda x: len(set(x))),
            count_unq_products=("product_id", lambda x: len(set(x))),
            count_by_products=("product_id", lambda x:  counter(x) ),
            sum_quantity=("quantity", np.sum),
            sum_price_unit=("price_unit", np.sum),
            sum_price_total=("price_total", np.sum),
            count_unq_countries=("country_id", lambda x: len(set(x))),
            prob_canceled=("is_canceled", np.mean)
        ).reset_index()
    )
    
    
    return df_customers


df_customers  = feature_engineering(df)
df_customers.head()

#Jak widzisz pojawiła się nowa kolumna `count_by_products`. W środku jest słownik. Zobaczmy dokładniej 👀.

df_customers["count_by_products"]

W praktyce to oznacza, że  `customer_id=0` kupił np. `product_id=0` - 89 razy, `product_id=1` - 73 razy, `product_id=3` - 68 razy itd.


Fajnie byłoby nam wypakować słownik do osobnych kolumn, na szczęście da się to zrobić bardzo łatwo. 
⌛️ Tylko poczekaj chwilkę na wykonanie.

df_count_products = df_customers["count_by_products"].apply(pd.Series).fillna(-1)
df_count_products.columns = ["product_{}".format(x) for x in df_count_products.columns]

df_count_products.head(5)

Mamy teraz osobny `dataframe` z 3878 kolumnami. Każda kolumna to jest `product_id`. Wartość -1 oznacza brak informacji,
czyli  klient nie kupował tego produktu, natomiast każda inna liczba (większa niż 0) oznacza, ile razy ten produkt 
był kupiony (to jest trochę optymistyczne zdanie, ale na razie możemy przyjąć, że jest ok).

#☝️ Zwróć uwagę, że pojawiła się linijka zmiany nazwy produktu. 
`df_count_products.columns = ["product_{}".format(x) for x in df_count_products.columns]` 
#W tym miejscu robimy bardzo prostą rzeczy, zamiast samego ID, np. 0 czy 2, dodajemy przedrostek, 
#aby kolumna nazywała się  `product_0` czy `product_2`. To nam przyda się za chwilę.


### Następny krok

Połączmy `df_count_products` z `df_customers` i wytrenujmy model.
Aby to połączyć użyjemy konstrukcji `concat()`. Wynikiem będzie to, że dokleimy to "z boku".
Innymi słowy liczba wierszy zostanie taka sama, liczba kolumn powiększy się o 3878 😱.
Model pewnie ucieszy się, będzie miał, co optymalizować.

Łączymy!

df_customers = pd.concat([df_customers, df_count_products], axis=1)
df_customers.shape

Tak jak spodziewaliśmy się, liczba wierszy została taka sama - 5879 (bo liczba wierszy to są klienci, nowych klientów nie przybyło). Liczba kolumn wzrosła nam do 3887. Możesz o tym pomyśleć, że teraz każdy klient posiada 3887 cech, czyli charakterystyk.

## 🤖 Czas na trenowanie modelu

Najpierw sprawdźmy, czy jakość modelu się nam poprawiła.

Uwaga! Model potrzebuje czasu na trenowanie, więc możesz spokojnie przez chwilę odpocząć. Model wykonuje "prostą"
i powtarzalną pracę, natomiast po Twojej stronie jest myślenie koncepcyjnie. 
Przyzwyczajaj się, to jest przyszłość, która już do nas przyszła :)

#df_customers = feature_engineering(df)
feats = get_feats(df_customers, black_list=["most_revenue_customer", "sum_price_total"])
X, y = get_X_y(df_customers, top_customers, feats)
model = xgb.XGBClassifier(max_depth=5, n_estimators=50, learning_rate=0.3)

train_and_get_scores(model, X, y)

Jakość modelu poprawiła się, mamy już ~85%. 

Przypomnę, że same drzewa decyzyjne dały nam 77%. XGBoost na podstawowych cechach dał nam 81% i teraz mamy 85%.

Może się wydawać, że to słabo, bo mieliśmy 99,9%. Chcę taki wynik znowu! 

Być może to jest przykra wiadomość, ale często model "stopuje" się właśnie w okolicy 60-70% i potem trzeba naprawdę
postarać się, aby wycisnąć więcej. No chyba, że użyjemy oczywistych zależności, ale to jest mało ciekawe.
Lepiej jednak pozwolić modelowi znaleźć coś, co może zaskoczyć.

Wytrenujmy model teraz niezależnie, aby można było zajrzeć i sprawdzić, co dla niego jest ważne.

model.fit(X, y)

eli5.show_weights(model, feature_names=feats, top=50)

#No teraz już przynajmniej mamy na co popatrzeć :). Widać, że ważność produktu `product_545` oraz `product_99` jest 
#ponad `count_orders`. Oczywiście, tak jak mówiłem, tutaj trzeba ostrożnie wyciągać wnioski. 
#Natomiast to, co jest jednoznaczne to to, że `product_id=545` zasłużył na to, aby zwrócić na niego uwagę 🥳. 
#To sprawdźmy :).

customer_ids_by_product = set(df[ df["product_id"] == 545 ]["customer_id"].unique())
len(customer_ids_by_product )

#Ten produkt kupiło nie tak wielu klientów, tylko 168. Sprawdźmy, ilu z nich należy do segmentu `most_revenue_customer`.

df_customers[ df_customers.customer_id.isin(customer_ids_by_product) ]["most_revenue_customer"].mean()

#No widzimy, że mamy 42% klientów, którzy kupili co najmniej `product_id=545` i należą do segmentu `most_revenue_customer`.
#Przypomnę, że normalnie mamy 22% klientów, którzy należą do segmentu `most_revenue_customer`.

df_customers["most_revenue_customer"].mean()

#Możemy na szybko postawić hipotezę (do zbadania) -  jeśli klient kupił `product_id=545` to prawdopodobieństwo, 
#że będzie należał do segmentu `most_revenue_customer` prawie podwaja się (od 23% do 42%). Brzmi jako "aha moment". 
#Tę hipotezę warto jeszcze zbadać dokładniej, ale jeżeli ona faktycznie działa, to oznacza, że warto sprawić, 
#aby więcej ludzi kupowało właśnie `product_id=545`.

#### Sprawdźmy, ile kosztuje ten produkt. Może to jakiś luksus ⚜️.

df[ df.product_id == 545 ]["price_unit"].value_counts()

Z ceną tego produktu bywa różnie, ale najczęściej cena jednostkowa, to jedynie 1,25 lub 3,9 😱. Taki ciekawy wniosek udało się znaleźć w tak łatwy sposób.

## Zadanie 5.1


Twoim zadaniem jest zrobić podobną analizę dla produktu.


### 💡 Podpowiedzi: 

1. Przygotuj `df_products`
2. Dodaj więcej cech, przed dodaliśmy `product_id`, ale możesz zrobić symetryczne lub wymyśl coś innego (może `order_id`) lub jeszcze coś innego, jest sporo pomysłów :) 
3. Wytrenuj model. 
4. Zobacz, co dla modelu było najbardziej istotne.

### YOUR CODE HERE

Jak **wykonasz** zadanie, należy zrobić:

1. Zrzut ekranu na którym widać rozwiązania, wtedy dostaniesz bonus :)
2. Wrzucić rozwiązanie na slacku do pokoju **[#dwthon_day5_done](https://bit.ly/3teY3l8)**
3. Dostać bonus za dobrą robotę 💪 i jutro będzie kolejna porcja.

*Uwaga! Jeśli masz problem z jakimś zadaniem, czegoś nie wiesz, to pamiętaj, że możesz zadawać pytania! Na tym polega nauka :) Pytania związane z 2 dniem DWthon zadawaj w kanale [#dwthon_day5](https://bit.ly/3qEBEMv)*.

## 🧠 Włączmy myślenie krytycznie


Ta cała nasza analiza była oparta na `price_total`, czyli przychodach. Co ze zwrotami? Być może nastąpił czas nieco bardziej wejść do króliczej nory?

Na co jeszcze warto zwrócić krytyczną uwagę?  Podpowiem, że jeszcze jest sporo rzeczy ;)

## 😇 Włączmy myślenie kreatywne

Pomyśl, jak możesz wykorzystać tę wiedzę, którą tutaj udało Ci się zdobyć?

Najlepiej, aby padły przykłady z Twojego podwórka. Jeśli nie masz (ale pomyśl, bo to tylko może się wydawać, że nie ma) to pomyśl o swoim otoczeniu.

Od razu powiem, że nie chodzi tylko o e-commerce. To może być dowolna branża. Reguła 80/20 jest uniwersalna. To samo dotyczy "aha-momentów".

## 🤝🗣️ Pomyśl o kooperacji i komunikacji

#### ☝️ Podziel się swoimi przemyśleniami na slacku **[#dwthon_day5_ideas](https://bit.ly/3qFcb5R)**. Jeśli masz pomysł, ale nie masz kodu - poproś na slacku o pomoc, aby ktoś pomógł Ci napisać kod. Natomiast jeśli masz kod, to koniecznie podziel się. Dzięki temu ktoś inny będzie w stanie łatwo powtórzyć i podzielić się swoimi przemyśleniami. Dzięki temu każdy zyskuje :).


%%html
<iframe style="height:500px;width:100%" src="https://www.youtube.com/embed/qC2MZcjGfcM" frameborder="0" allow="autoplay; encrypted-media" allowfullscreen></iframe>

## Twój Feedback i delta rozwoju 

Daj mi znać koniecznie, co sprawiło Ci przyjemność, co trudność oraz czego udało się nauczyć w tej krótkiej [ankiecie](https://bit.ly/3rEnOLD). Pozwoli mi ona ulepszyć zadania, a Tobie lepiej trakować swój rozwój: win-win 😇

### 🎁 Opublikuj rozwiązania, czeka na Ciebie bonus 

1. Wrzucić rozwiązanie na slacku do pokoju [#dwthon_day5_done](https://bit.ly/3teY3l8)
2. Otrzymasz bonusy, a w nich m.in. kolejną lekcję wideo z naszego najlepszego programu nauki [DS & ML w praktyce](
https://dataworkshop.eu/pl/practical-machine-learning)
3. Umiesz więcej ;) 

## 💪 Pochwal się światu, że DWthon za Tobą 💪 

Udało Ci się! Jesteś na mecie, koniecznie powiedz o tym innym. Być może kogoś to zmotywuje do działania, a dodatkowo warto mówić głośno o nowych umiejętnościach i doświadczeniach, bo nigdy nie wiesz, jaka furtka może się otworzyć ;) 

Pamiętaj o #dwthon #hackoutsidethebox ;) 

Śmiało nas oznaczaj, będzie nam miło :))
